In [ ]:
from __future__ import division

import os
import zipfile

In [ ]:
teamname = 'Machine_Contact_Enterprise_SETI'
mydatafolder = os.path.join( os.environ['PWD'], teamname )      
input_png_folder = mydatafolder + '/cleanPngsTestFinal'
output_zip_folder = mydatafolder + '/zippedPngsTestFinal_4'
workingIndexFile = os.path.join( mydatafolder + '/public_list_basic_test_hackathon_final.csv' )

In [ ]:
#open(workingIndexFile).read().split(',')

In [ ]:
ls -lah $mydatafolder/zippedPngsTestFinal_1

# Create Test .zip Files

In [ ]:
indexfile_rows = open(workingIndexFile).readlines()
                                                    
uuids_classes_as_list = indexfile_rows  [1:]#slice off the first line (header)
  
def row_to_json(row):
    uuid = row.strip('\n')  #strip \n and split uuid
    return {'uuid':uuid}


uuids_classes_as_list = map(lambda row: row_to_json(row), uuids_classes_as_list)
print "found {} files".format(len(uuids_classes_as_list))

In [ ]:
#training_percentage = 0.00
#test_percentage = 1.00

#assert training_percentage + test_percentage <= 1.0

#training_set_group_by_class = {}
#test_set_group_by_class = {}
#for k, v in uuids_group_by_class.iteritems():
    
#    total = len(v)
#    training_size = int(total * training_percentage)
#    test_size = int(total * test_percentage)
#    
#    training_set = v[:training_size]
#    test_set = v[-1*test_size:]
#    
#    training_set_group_by_class[k] = training_set
#    test_set_group_by_class[k] = test_set
#    
#    print '{}: training set size: {}'.format(k, len(training_set))
#    print '{}: test set size: {}'.format(k, len(test_set))

In [ ]:
if os.path.exists(output_zip_folder) is False:
    os.makedirs(output_zip_folder)

In [ ]:
# Create the Zip files containing the test PNG files using the following naming convention:
# testset_<NUMBER>_<CLASS>.zip (step 4 will break if a different naming convention is used)
# Refer to https://www.ibm.com/watson/developercloud/visual-recognition/api/v3/#classify_an_image for ZIP size and content limitations:
# "The max number of images in a .zip file is limited to 20, and limited to 5 MB."
fnames = [input_png_folder + '/' + vv['uuid'] + '.dat.png' for vv in uuids_classes_as_list]

# archive counter
count = 1
# number of image files in archive counter
image_count = 0
for fn in fnames:

    archive_name = '{}/testset_{}.zip'.format(output_zip_folder, count)

    if os.path.exists(archive_name):
        if os.path.getsize(archive_name) + os.path.getsize(fn) >= 4.8 * 1024 ** 2:
            # current ZIP archive size + size of this file > max size (or at least close to); create new archive
            count += 1
            image_count = 0
            archive_name = '{}/testset_{}.zip'.format(output_zip_folder, count)
            print 'creating new archive', archive_name
            zz = zipfile.ZipFile(archive_name, mode='w')
        else:
            zz = zipfile.ZipFile(archive_name, mode='a')
    else:
        print 'creating new archive', archive_name
        zz = zipfile.ZipFile(archive_name, mode='w')

    zz.write(fn)
    zz.close()

    image_count += 1
    # the number of files > max number of files supported by API; create new archive
    if image_count > 19:
        count +=1
        image_count = 0